# FORECAST в Google Colab: Новости → Тикеры → Влияние на свечи

Инструкции:
- Загрузите ваш проект в Google Drive в папку `MyDrive/news_analize` (вся папка с .py и .csv).
- В меню Colab включите GPU: Runtime → Change runtime type → Hardware accelerator → GPU.
- Запускайте ячейки сверху вниз.


In [ ]:
# Маунт Google Drive
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = '/content/drive/MyDrive/news_analize'
%cd $PROJECT_DIR
!ls -la


In [ ]:
pip install -q pandas==2.2.2 numpy==1.26.4 pyarrow==16.1.0 scikit-learn==1.5.1 razdel==0.5.0 pymorphy3==2.0.2 pymorphy3-dicts-ru==2.4.417150.4580142 rapidfuzz==3.9.6 emoji==2.12.1 text-unidecode==1.3 torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [ ]:
# Авторазметка тикеров (если нет столбца tickers)
!python auto_label_tickers.py --news test_news.csv --out task_1_news_autolabeled.csv

import pandas as pd
x = pd.read_csv('task_1_news_autolabeled.csv')
print(x[['publish_date','title','tickers']].head())


In [ ]:
# Обучение модели новости→тикеры (щадящие параметры)
!python train_news_ticker.py --news task_1_news_autolabeled.csv --artifacts artifacts/ --epochs 3 --batch_size 64 --max_len 256

import os
print('Artifacts:', os.listdir('artifacts'))


In [ ]:
# Инференс влияния на свечи
# Замените имя файла свечей при необходимости
!python infer_news_to_candles.py \
  --news task_1_news_autolabeled.csv \
  --candles public_test_candles.csv \
  --artifacts artifacts/ \
  --out nn_features.parquet

import pandas as pd
fe = pd.read_parquet('nn_features.parquet')
print(fe.head())
print('rows:', len(fe))
